In [1]:
%load_ext autoreload
%autoreload 1
%aimport rails,aise

In [2]:
import torch
import torch.nn as nn
from collections import deque
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import model as _model
import utils as _tools

from resnet import ResNet18

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import time

"""
initialize the trainned model
"""

start = time.perf_counter()

resnet = _model.TransferModel(use_cpu=False)

resnet.model.load_state_dict(torch.load("models/Cardiomegaly_resnet18.pth"))
resnet.model.to(DEVICE)
resnet.model.eval()

end = time.perf_counter()

print("Done within {} secs.".format(end-start))

Done within 28.292062821798027 secs.


In [5]:
model = ResNet18(num_classes=2)
model.load_state_dict(torch.load("./models/chexpert_resnet18.pt"))
model.to(DEVICE)
model.eval()

ResNet(
  (feature): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu):

In [6]:
# the training data from the data loader
n_rows = 1000 # how many rows to sample
row_count = 0
x = []
y = []
for image, labels in resnet.dataloader_train.dataset:
    row_count += 1
    x.append(image)
    y.append(labels[0]) # the label for cadiomegaly
    
    if row_count == n_rows:
        break
    
x_train = torch.FloatTensor(x)
y_train = torch.LongTensor(y)

print("done")

done


In [7]:
# the testing data from the data loader
n_rows = 300 # how many rows to sample
row_count = 0
x = []
y = []
for image, labels in resnet.dataloader_valid.dataset:
    row_count += 1
    x.append(image)
    y.append(labels[0]) # the label for cadiomegaly
    
    if row_count == n_rows:
        break
    
x_test = torch.FloatTensor(x)
y_test = torch.LongTensor(y)

print("done")

done


In [8]:
print(
    "ResNet18 accuracy: {}".format(
    (model(x_test.to(DEVICE)).max(dim=1)[1]==y_test.to(DEVICE)).float().mean().item()
))

ResNet18 accuracy: 0.7179487347602844


In [9]:
CONFIG = {
    "start_layer": 1,
    "n_class": 2,
    "aise_params": [
        {"hidden_layer": 2, 
         "sampling_temperature": 1, 
         "max_generation": 20, 
         "n_neighbors" : 5,
         "n_population" : 2 * 5 * 10,
         "mut_range": (.005, .015)},
    ]
}

rails_clf = rails.RAILS(model, 
              CONFIG, 
              x_train,
              y_train)

5


In [10]:
y_proba = rails_clf.predict(x_test)
y_pred = y_proba.argmax(axis=1)
(y_pred==y_test.numpy()).astype("float").mean()

0.7136752136752137

In [11]:
print("Recall: {}".format(
    ((y_pred==y_test.numpy()) * (y_pred==1)).sum()/(y_test.numpy()==1).sum()
))
print("Precision: {}".format(
    ((y_pred==y_test.numpy()) * (y_pred==1)).sum()/(y_pred==1).sum()
))

Recall: 0.058823529411764705
Precision: 0.5714285714285714


In [12]:
import pandas as pd

df = pd.DataFrame({
    "y_true": y_test.numpy(),
    "y_pred": y_pred,
    "y_prob": y_proba[:,1]
})

_tools.get_classification_metrics(df)

,0
optimal_threshold,0.400000
true negatives,162.000000
true positives,5.000000
false positives,4.000000
false negatives,63.000000
sensitivity,0.073529
specificity,0.975904
F1-score,0.129870
precision,0.555556
recall,0.073529


In [13]:
np.array(_tools.get_classification_metrics(df)[1:5][0])[[0,2,3,1]].reshape(2,2)

array([[162.,   4.],
       [ 63.,   5.]])